<a href="https://colab.research.google.com/github/bposantos/Circular_Dichroism_Analysis/blob/main/cd_deconvolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CD Deconvolution

##Pre-processing

In [ ]:
import pandas as pd

In [ ]:
# Necessary libraries
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

In [ ]:
# User authentication
auth.authenticate_user()

# Credentials
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Google Sheets file opening
sheet_name = 'worksheet'
sheet = gc.open(sheet_name)

# Pages opening inside the file
# Suggestion: put the content of each subplot in each page
page_name1 = 'data_01'
page1 = sheet.worksheet(page_name1)

In [ ]:
# Obtaining pages values
data1 = page1.get_all_values()

In [ ]:
# Pandas dataframe creation
df1 = pd.DataFrame(data1[1:], columns=data1[0])

In [ ]:
# Replace commas with dots in all columns (ONLY IF NECESSARY)
df1 = df1.replace(',', '.', regex=True)

## Deconvolution

In [ ]:
#convert string to float
df1 = df1.apply(pd.to_numeric, errors='coerce')

In [ ]:
#############################################################################################################################################
#Protein Information
#############################################################################################################################################

#molecular weight in daltons
molecular_weight = 1000 #change according to the molecular weight

#number of residues
aa = 13 #change according to the number of amino acids

#cuvette path length in mm
path_length = 0.1 #change according to the path length

#Molarity in micromolar
molarity = 100 #change according to the molarity of the sample

In [ ]:
#applying the ellipticity function
def multiply(x,y):
	return round(x*y,3)

In [ ]:
#The mean residue weight of a peptide is the molecular weight divided by the number of backbone amides (number of amino acids -1 if the protein is not acetylated). It is ~ 115 for most proteins if the molecular weight of the sample is not known.
mean_residue_weight = (molecular_weight/aa-1)

#concentration in mg/ml is molarity (M) x Molecular Weight (Da)
concentration = round(molarity*molecular_weight*0.000001,3)

#Ellipticity, [θ], in deg. · cm2/dmol = (millidegrees times mean residue weight) / (path length in mm times concentration in mg/ml)Norma J. Greenfield Nat Protoc. 2006
ellipticity=round(mean_residue_weight/(path_length*concentration),2)

# Apply the multiplication to all columns except 'Wavelength'
df1.loc[:, df1.columns != 'Wavelength'] = df1.loc[:, df1.columns != 'Wavelength'].applymap(lambda x: multiply(x, ellipticity))


In [ ]:
#Create a new page to insert the deconvoluted data
page_01 = sheet.add_worksheet(title='worksheet_deconv',rows=df1.shape[0]+1, cols=df1.shape[1])

In [ ]:
#Update the new page with the deconvoluted data
data = [df1.columns.values.tolist()] + df1.values.tolist()
page_01.update('A1', data)